In [ ]:
# %pip install tokenizers

In [2]:
from datasets import load_dataset
dataset=load_dataset('parquet',data_files=["data/data.parquet"], streaming=True)['train']
dataset


IterableDataset({
    features: ['code', 'repo_name', 'path', 'language', 'license', 'size'],
    num_shards: 1
})

In [ ]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

from transformers import PreTrainedTokenizerFast

tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
tokenizer.decoder=decoders.ByteLevel()
tokenizer.clean_up_tokenization_spaces=True

special_tokens = ["<|pad|>", "<|mask|>", "<|cls|>", "<|cs|>", "<|ts|>", "<|css|>"]
tokenizer.add_special_tokens(special_tokens)
trainer = trainers.BpeTrainer(vocab_size=5000, special_tokens=special_tokens,max_token_length=10, show_progress=True)

def datasetGenerator():
    for row in dataset.take(500):
        yield row['code']

tokenizer.train_from_iterator(datasetGenerator(), trainer=trainer)

hft=PreTrainedTokenizerFast(pad_token='<|pad|>', cls_token='<|cls|>', mask_token='<|mask|>',sep_token='<|pad|>', tokenizer_object=tokenizer )
hft.save_pretrained('data/bert')
tokenizer=hft

In [4]:
ids=tokenizer("foo.bar(123); hello world")["input_ids"]
print(ids)
print(tokenizer.convert_ids_to_tokens(ids))
tokenizer.decode(ids)


[75, 1523, 19, 2746, 13, 1536, 24, 200, 848, 300, 325, 389, 2546]
['f', 'oo', '.', 'bar', '(', '12', '3', ');', 'Ġh', 'el', 'lo', 'Ġw', 'orld']


'foo.bar(123); hello world'